In [1]:
from obsidianScripts.main import main
import json
from gptStuff.masterGPT import GPTFlow

json_data = None
with open('obsidianScripts/testJSONS/test1.json', 'r') as file:
    json_data = file.read()
    file.close()
json_data=json.loads(json_data)


main(json_data=json_data,debug=True)

safemode is on
this will not write to the files
debug mode is on
{'type': 'memory', 'person': 'Harshini', 'location': {'0': 'Important Details', '1': 'Relationship'}, 'item': 'Roommates with Ananya'}
Harshini Kundurthi
attribute classifier dict_keys(['Likes and Dislikes', 'Important Details', 'Gift Ideas', 'Birthday', 'Dietary Restrictions', 'pronouns', 'tags'])
first location Important Details
['1']
['1']
**Relationship:** 
*Elaborate if you want!*
here
file written


In [2]:
# get the token from the json
with open('token.json', 'r') as file:
    token = file.read()
    file.close()
token = json.loads(token)
token = token['token']
print(token)

7266721080:AAGBaAytkmacRQgLs8DWp271Fh3wqRSDL6U


In [3]:
import requests

def set_webhook(bot_token, webhook_url):
    url = f'https://api.telegram.org/bot{bot_token}/setWebhook'
    payload = {
        'url': webhook_url
    }
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        print('Webhook set successfully!')
    else:
        print('Failed to set webhook.')
        print('Error:', response.text)

# Replace <YOUR_BOT_TOKEN> with your actual bot token
webhook_url = 'https://de50-140-82-220-240.ngrok-free.app'+'/telegram-webhook'

set_webhook(token, webhook_url)

Webhook set successfully!


In [4]:
from flask import Flask, request
import subprocess

import requests
app = Flask(__name__)


@app.route('/telegram-webhook', methods=['POST'])
def webhook():
    # Get the input data from the request
    input_data = request.json

    # Extract the relevant information from the input data
    try:
        message_text = input_data['message']['text']
        chat_id = input_data['message']['chat']['id']
    except:
        print('Invalid input data:', input_data)
        return 'OK'

    # Run the Python script with the input data
    result = run_python_script(message_text)

    # Send the result back to Telegram
    send_message(chat_id, result)

    return 'OK'

def run_python_script(input_text):
    # Run the Python script as a subprocess and capture the output
    try:
        response = GPTFlow(input_text)
    except Exception as e:
        response = f'An error occurred: {e}'

    return response

def send_message(chat_id, text):
    # Send a message back to Telegram using the sendMessage method
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': text
    }
    requests.post(url, json=payload)

if __name__ == '__main__':
    print('Starting the Flask app')
    print('token:',token)
    app.run(port=3000)

Starting the Flask app
token: 7266721080:AAGBaAytkmacRQgLs8DWp271Fh3wqRSDL6U
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


inside parent Test
{'sister': {'details': "Please provide more information about the task you'd like us to assist with."}, 'brother': {'context': 'Awaiting further details to execute the task effectively.'}}
sister result {'type': 'retrieval', 'item': 'task'}
safemode is on
this will not write to the files


127.0.0.1 - - [19/Jun/2024 22:57:03] "POST /telegram-webhook HTTP/1.1" 200 -


inside parent Harshini is who
{'sister': {'details': 'Retrieve information about Harshini'}, 'brother': {'context': 'Inform the user about who Harshini is based on the information retrieved'}}
sister result {'type': 'retrieval', 'person': 'Harshini', 'location': {}}
safemode is on
this will not write to the files


...

127.0.0.1 - - [19/Jun/2024 22:59:23] "POST /telegram-webhook HTTP/1.1" 200 -


inside parent Who's harshini
{'sister': {'details': 'Retrieve information about Harshini including tags, pronouns, and birthday'}, 'brother': {'context': 'Inform the boss about the details of Harshini including tags, pronouns, and birthday'}}
sister result {'type': 'list', 'content': [{'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Tags'}}, {'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Pronouns'}}, {'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Birthday'}}]}
safemode is on
this will not write to the files
safemode is on
this will not write to the files
safemode is on
this will not write to the files

<Harshini>
tags:tags:
  - CoppenhagenInterpretationOfObjectPermanence
  - SAAYA
  - bollywoodDance
pronouns:pronouns: she/her
Birthday:Birthday: 2005-11-09
brotherPrompt Inform the boss about the details of Harshini including tags, pronouns, and birthday
<informationForBrother>
<Harshini>
tags:tags:
  - CoppenhagenInterpretationOfObjectPerman

127.0.0.1 - - [19/Jun/2024 23:00:13] "POST /telegram-webhook HTTP/1.1" 200 -


inside parent Who's harshini
{'sister': {'details': 'Retrieve information about Harshini including tags, pronouns, and birthday'}, 'brother': {'context': 'Inform the boss about the retrieved details of Harshini'}}
sister result {'type': 'list', 'content': [{'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Tags'}}, {'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Pronouns'}}, {'type': 'retrieval', 'person': 'Harshini', 'location': {'0': 'Birthday'}}]}
safemode is on
this will not write to the files
safemode is on
this will not write to the files
safemode is on
this will not write to the files

<Harshini>
tags:tags:
  - CoppenhagenInterpretationOfObjectPermanence
  - SAAYA
  - bollywoodDance
pronouns:pronouns: she/her
Birthday:Birthday: 2005-11-09
brotherPrompt Inform the boss about the retrieved details of Harshini
<informationForBrother>
<Harshini>
tags:tags:
  - CoppenhagenInterpretationOfObjectPermanence
  - SAAYA
  - bollywoodDance
pronouns:pronouns: she/h

127.0.0.1 - - [19/Jun/2024 23:00:29] "POST /telegram-webhook HTTP/1.1" 200 -
